In [ ]:
!pip install datasets transformers torch

In [ ]:
from datasets import load_dataset

dataset_train = load_dataset(
    'cifar10',
    split='train'
    # The 'cifar10' loader doesn't support 'ignore_verifications'. Removing it.
)

dataset_train

In [ ]:
dataset_test = load_dataset(
    'cifar10',
    split='test', # training dataset
    # ignore_verifications=True  # set to True if seeing splits Error
)

dataset_test

In [ ]:
num_classes = len(set(dataset_train['label']))
labels = dataset_train.features['label']
num_classes, labels

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(X=dataset_train[10]['img'])

In [ ]:
dataset_train[0]['label'],labels.names[dataset_train[10]['label']]

In [ ]:
from transformers import ViTFeatureExtractor
model_id = 'google/vit-base-patch16-224-in21k'
feature_extractor = ViTFeatureExtractor.from_pretrained(
    model_id
)

In [ ]:
feature_extractor

In [ ]:
example = feature_extractor(
    dataset_train[0]['img'],
    return_tensors='pt'
)
example

In [ ]:
example['pixel_values'].shape

In [ ]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
def preprocess(batch):
    inputs = feature_extractor(
        batch['img'],
        return_tensors='pt'
    )
    inputs['label'] = batch['label']
    return inputs

In [ ]:
prepared_train = dataset_train.with_transform(preprocess)
prepared_test = dataset_test.with_transform(preprocess)

In [ ]:
def collate_fn(batch):
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        'labels': torch.tensor([x['label'] for x in batch])
    }

In [ ]:
!pip install datasets evaluate

In [ ]:
import numpy as np
from evaluate import load # import load instead of load_metric
metric = load("accuracy") # use load instead of load_metric
def compute_metrics(p):
    return metric.compute(
        predictions=np.argmax(p.predictions, axis=1),
        references=p.label_ids
    )

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="./cifar",
  per_device_train_batch_size=16,
  evaluation_strategy="steps",
  num_train_epochs=4,
  save_steps=100,
  eval_steps=100,
  logging_steps=10,
  learning_rate=2e-4,
  save_total_limit=2,
  remove_unused_columns=False,
  push_to_hub=False,
  load_best_model_at_end=True,
)

In [ ]:
from transformers import ViTForImageClassification

labels = dataset_train.features['label'].names

model = ViTForImageClassification.from_pretrained(
    model_id,
    num_labels=len(labels)
)

In [ ]:
model.to(device)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=prepared_train,
    eval_dataset=prepared_test,
    tokenizer=feature_extractor,
)

In [ ]:
metrics = trainer.evaluate(prepared_test)
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

In [ ]:
image = dataset_test["img"][10].resize((200,200))
image

In [ ]:
actual_label = dataset_test["label"][10]

labels = dataset_test.features['label']
actual_label, labels.names[actual_label]

In [ ]:
from transformers import ViTForImageClassification, ViTFeatureExtractor
model_name_or_path = 'LaCarnevali/vit-cifar10'
model_finetuned = ViTForImageClassification.from_pretrained(model_name_or_path)
feature_extractor_finetuned = ViTFeatureExtractor.from_pretrained(model_name_or_path)

In [ ]:
inputs = feature_extractor_finetuned(image, return_tensors="pt")

with torch.no_grad():
    logits = model_finetuned(**inputs).logits

In [ ]:
predicted_label = logits.argmax(-1).item()
labels = dataset_test.features['label']
labels.names[predicted_label]